In [53]:
# Tutorial: https://github.com/terrier-org/cikm2021tutorial/tree/main
import os
import pandas as pd
import pyterrier as pt

In [54]:
if not pt.started():
    pt.init()

In [55]:
#dataset = pt.datasets.get_dataset("msmarco_document")
dataset = pt.datasets.get_dataset("vaswani")


In [56]:
for i, d in enumerate(dataset.get_corpus_iter()):
    print (d)
    if (i > 3):
        break


{'docno': '1', 'text': 'compact memories have flexible capacities  a digital data storage\nsystem with capacity up to bits and random and or sequential access\nis described'}
{'docno': '2', 'text': 'an electronic analogue computer for solving systems of linear equations\nmathematical derivation of the operating principle and stability\nconditions for a computer consisting of amplifiers'}
{'docno': '3', 'text': 'electronic coordinate transformer  circuit details are given for\nthe construction of an electronic calculating unit which enables\nthe polar coordinates of a vector modulus and cosine or sine of the\nargument to be derived from those of a rectangular system of axes'}
{'docno': '4', 'text': 'the british computer society  report of a conference held in cambridge\njune'}
{'docno': '5', 'text': 'millimicrosecond digital computer logic  a system of fast pulse\nlogic is described which combines the efficiency of transformer coupled\nstages with digit delay tolerances approaching that

In [69]:
index_path = "./vaswani_index"

In [70]:
indexer = pt.TRECCollectionIndexer(index_path, blocks=True)
indexref = indexer.index(dataset.get_corpus())

In [71]:
indexref.toString()

'./vaswani_index/data.properties'

In [72]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 0
Number of tokens: 271581
Field names: []
Positions:   true



In [100]:
lex = index.getLexicon()

for termid in range(7750, 7756):
    lee = lex.getLexiconEntry(termid)
    print (termid, lee.getKey())

7750 geocyclotron
7751 incur
7752 climatolog
7753 ephi
7754 handlett
7755 sansserif


In [128]:
# Posting list del término ´x´
meta = index.getMetaIndex()
inv  = index.getInvertedIndex()
#
le = lex.getLexiconEntry("run")


list_docs  = []
list_freqs = []
# the lexicon entry is also our pointer to access the inverted index posting list
for posting in inv.getPostings(le):
    docno = meta.getItem("docno", posting.getId())
    #print("(%s, %d), " % (docno, posting.getFrequency()), end = '')
    list_docs.append(int(docno))
    list_freqs.append(int(posting.getFrequency()))
#
print (list_docs)
print (list_freqs)   

[948, 950, 2863, 3164, 3462, 3496, 4898, 4953, 5071, 5538, 5667, 5815, 6711, 6829, 6862, 6942, 6991, 7436, 7870, 8557, 8959, 9236, 9778, 9974, 10159, 10857]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2]


In [92]:
# Términos en un doc
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 101 #docids are 0-based
#NB: postings will be null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print("%s with frequency %d" % (lee.getKey(),posting.getFrequency()))

electron with frequency 2
given with frequency 1
measur with frequency 1
densiti with frequency 1
compon with frequency 1
squar with frequency 1
frequenc with frequency 1
analys with frequency 1
determin with frequency 1
signal with frequency 1
rate with frequency 1
shift with frequency 1
excess with frequency 1
wave with frequency 2
observ with frequency 1
term with frequency 1
ordinari with frequency 1
valu with frequency 1
satellit with frequency 2
artifici with frequency 2
ionospher with frequency 2
radio with frequency 1
distribut with frequency 1
fade with frequency 4
appar with frequency 1
doppler with frequency 1
differ with frequency 1
upper with frequency 1
forth with frequency 1
yield with frequency 1
content with frequency 1
extraordinari with frequency 1
outlin with frequency 1
procedur with frequency 1
vertic with frequency 1
invers with frequency 1
faradai with frequency 4
exactli with frequency 1
proport with frequency 1
explan with frequency 1


In [74]:
lexicon = index.getLexicon()

In [75]:
len(lexicon)

7756

In [76]:
print (lexicon)

<org.terrier.structures.Lexicon at 0x7ff9dcc51260 jclass=org/terrier/structures/Lexicon jself=<LocalRef obj=0x56070163a6a0 at 0x7ff9dcc8a7f0>>


In [83]:

print (lexicon.getIthLexiconEntry(3))

<java.util.Map$Entry at 0x7ff9dc5e62f0 jclass=java/util/Map$Entry jself=<LocalRef obj=0x56070163a6f8 at 0x7ff9dcc62610>>


In [104]:
# Probar compresion
# https://github.com/searchivarius/PyFastPFor/blob/master/python_bindings/examples.ipynb

In [105]:
!pip install pyfastpfor


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for pyfastpfor: filename=pyfastpfor-1.3.6-cp310-cp310-linux_x86_64.whl size=428464 sha256=cecfe0486d67e0eea3801b65123567aafdb8f7503db3ca97100a0feb55df464a
  Stored in directory: /home/jovyan/.cache/pip/wheels/10/61/2a/0a26b0c266984d86decd330a4a4f96c3f2a453e5bdeb772c8c
Successfully built pyfastpfor


In [106]:
from pyfastpfor import *
import numpy as np
# Get the list of all codecs
getCodecList()

['BP32',
 'copy',
 'fastbinarypacking16',
 'fastbinarypacking32',
 'fastbinarypacking8',
 'fastpfor128',
 'fastpfor256',
 'maskedvbyte',
 'newpfor',
 'optpfor',
 'pfor',
 'pfor2008',
 'simdbinarypacking',
 'simdfastpfor128',
 'simdfastpfor256',
 'simdgroupsimple',
 'simdgroupsimple_ringbuf',
 'simdnewpfor',
 'simdoptpfor',
 'simdpfor',
 'simdsimplepfor',
 'simple16',
 'simple8b',
 'simple8b_rle',
 'simple9',
 'simple9_rle',
 'simplepfor',
 'streamvbyte',
 'varint',
 'varintg8iu',
 'varintgb',
 'vbyte',
 'vsencoding']

In [116]:
arrSize = 128 * 28
maxVal = 2048
# 1. Example without data differencing

# All arrays the library use must be contiguous-memory C-style numpy arrays
inp = sorted(np.array(np.random.randint(0, maxVal, arrSize), dtype = np.uint32, order = 'C'))
inpCompDecomp = np.zeros(arrSize, dtype = np.uint32, order = 'C')

# To be on the safe side, let's reserve plenty of additional memory:
# sometimes the size of compressed data is not smaller than the size 
# of the original one
inpComp = np.zeros(arrSize + 1024, dtype = np.uint32, order = 'C')

# Obtain a codec by name
#codec = getCodec('simdbinarypacking')
codec = getCodec('pfor')

# Compress data
compSize = codec.encodeArray(inp, arrSize, inpComp, len(inpComp))
 
print('Compression ratio: %g' % (float(compSize)/arrSize))

# Decompress data
assert(arrSize == codec.decodeArray(inpComp, compSize, inpCompDecomp, arrSize))
assert(np.all(inpCompDecomp == inp))

Compression ratio: 0.3524


In [117]:
print (arrSize, compSize)

3584 1263


In [118]:
print (len(inpCompDecomp))

3584


In [182]:
def compute_dgaps(docids):
    refval = docids[0]
    dgaps  = [refval]
    for cur in range(1, len(docids)):
        this_gap = docids[cur] - refval
        #print (cur, refval, docids[cur], this_gap)
        refval   = docids[cur]
        dgaps.append(this_gap)
    return dgaps

In [183]:
# Comprimir list_docs y list_freqs (vienen de arriba)

list_docs_size = len(list_docs)

# To be on the safe side, let's reserve plenty of additional memory: sometimes
# the size of compressed data is not smaller than the size of the original one
list_docs_comp   = np.zeros(list_docs_size + 1024, dtype = np.uint32, order = 'C')
list_docs_decomp = np.zeros(list_docs_size, dtype = np.uint32, order = 'C')

# Obtain a codec by name
codec = getCodec('simdbinarypacking')
#codec = getCodec('pfor')
codec = getCodec('vbyte')


# Compress data
list_docs_comp_size = codec.encodeArray(list_docs, list_docs_size, list_docs_comp, len(list_docs_decomp))
 
print('Compression ratio: %g' % (float(list_docs_comp_size) / list_docs_size))

# Decompress data
assert(list_docs_size == codec.decodeArray(list_docs_comp, list_docs_comp_size, list_docs_decomp, list_docs_size))
assert(np.all(list_docs_decomp == list_docs))









Compression ratio: 0.5


In [184]:
print (float(list_docs_comp_size))

13.0


In [185]:
# Compute dgaps
list_docs_dgaps = compute_dgaps(list_docs)

In [186]:
# Compress data
list_docs_comp_size2 = codec.encodeArray(list_docs_dgaps, list_docs_size, list_docs_comp, len(list_docs_decomp))
 
print('Compression ratio: %g' % (float(list_docs_comp_size2) / list_docs_size))


Compression ratio: 0.423077


In [187]:
list_docs_1 = sorted(np.array(np.random.randint(1, 2, list_docs_size), dtype = np.uint32, order = 'C'))


In [188]:
list_docs_1

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [189]:
list_docs_comp_size3 = codec.encodeArray(list_docs_1, list_docs_size, list_docs_comp, len(list_docs_decomp))
 
print('Compression ratio: %g' % (float(list_docs_comp_size3) / list_docs_size))


Compression ratio: 0.269231


In [190]:
print (float(list_docs_comp_size2))
print (float(list_docs_comp_size3))
print (float(list_docs_size))

11.0
7.0
26.0


In [191]:
# Pibiri
# https://github.com/jermp/data_compression_course


# Compression ratio: ratio of the size of the compressed data to the size of the original data. 
# The higher the compression ratio, the more space is saved by compression.
